# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [2]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
ori_data = np.load('../OUTPUT/test/samples/evening_peak_etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/test/ddpm_fake_evening_peak_etth_milestone_360.npy')

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [ ]:
for j in range(10):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# doube input, ep, 0.65


/mnt/data728/duyin/anaconda3/envs/torch201/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Iter 0:  context-fid = 0.5587078022792721 

Iter 1:  context-fid = 0.6253669329135995 

Iter 2:  context-fid = 0.7053471196392455 

Iter 3:  context-fid = 0.7902510481001057 

Iter 4:  context-fid = 0.5882294264902405 

Final Score:  0.6535804658844927 ± 0.11685209208770973
Iter 0:  context-fid = 0.5272431748964616 

Iter 1:  context-fid = 0.8284702200914066 

Iter 2:  context-fid = 0.6234272503971788 

Iter 3:  context-fid = 0.6849564110538824 

Iter 4:  context-fid = 0.6511339024792067 

Final Score:  0.6630461917836271 ± 0.13601632762747373
Iter 0:  context-fid = 0.7256012849266877 

Iter 1:  context-fid = 0.6399291726956633 

Iter 2:  context-fid = 0.502985731103273 

Iter 3:  context-fid = 0.5861183248789517 

Iter 4:  context-fid = 0.5166814442365177 

Final Score:  0.5942631915682186 ± 0.11405736141077025
Iter 0:  context-fid = 0.6871807016860628 

Iter 1:  context-fid = 0.4962259209427571 

Iter 2:  context-fid = 0.5650345049803356 

Iter 3:  context-fid = 0.5553992649738113 



## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [4]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [5]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.20556674196073937 

Iter 1:  cross-correlation = 0.20440501776632192 

Iter 2:  cross-correlation = 0.21036383697730177 

Iter 3:  cross-correlation = 0.21056121149998147 

Iter 4:  cross-correlation = 0.18618610341975078 

Final Score:  0.20341658232481907 ± 0.01244454091592246
